# LLM Society: Visualization Demo (Groups, Centrality, Intervention)

This notebook demonstrates:
- Group mean beliefs over time (by persona attribute)
- Centrality vs final belief/exposure scatter
- Scheduled intervention that reduces talk probability for selected nodes

All comments are in English. Information texts are in English as requested.



In [ ]:
# Imports
from llm_society import network
from llm_society import viz  # optional, we will mainly use net.plot(plot_type=...)

# Standard scientific stack
import numpy as np
import matplotlib.pyplot as plt



In [ ]:
# Build a small network with two political segments and a scheduled intervention

# Information is in English
information_text = "Increasing gas taxes will significantly reduce inflation."

# Two political segments to create visible group differences
segments = [
    {"proportion": 0.5, "traits": {"political": {"choices": {"Democrat": 0.85, "Republican": 0.15}}}},
    {"proportion": 0.5, "traits": {"political": {"choices": {"Democrat": 0.15, "Republican": 0.85}}}},
]

net = network(
    information=information_text,
    n=24, degree=4, rounds=10,
    depth=0.5, depth_max=4, edge_frac=0.5,
    seeds=[0,1], seed_belief=0.95, talk_prob=0.5,
    mode="llm", rng=0, model="gpt-4.1",
    segments=segments,
    # Start an intervention at round 6 on a subset of nodes
    intervention_round=6,
    intervention_nodes=[2,5,7,11,13,17],
    intervention_talk_prob_multiplier=0.5,
)

# Run the simulation; this will trigger LLM calls if model is live
net.simulate()
print(f"History length (rounds+1): {len(net.history)}")



In [ ]:
# 1) Group mean beliefs over time (by persona attribute, default: political)
# This uses the new 'type' dispatch on net.plot()
net.plot(type="group_beliefs", attr="political")



In [ ]:
# 2) Centrality vs final belief/exposure scatter
# Try different metrics: "degree", "betweenness", "eigenvector"
net.plot(type="centrality", metric="betweenness")



In [ ]:
# 3) Intervention effect on coverage (vertical marker at intervention round)
# If you omit intervention_round here, it is auto-detected from history metadata.
net.plot(type="intervention_effect", intervention_round=6)



In [ ]:
# (Optional) Standard coverage plot and final beliefs heat map for context
net.plot(type="coverage")
net.plot(type="final_beliefs")



In [ ]:
# (Optional) Animation (may be slower and require a JS-enabled notebook UI).
# You can also save it via net.plot(type="animation", save="anim.gif").
net.plot(type="animation")



In [ ]:
# (Optional) Using a custom NetworkX graph (e.g., from real data)
# Nodes should preferably be integers 0..N-1; if not, they are relabeled internally.

import networkx as nx

# Example: start from a small real-like structure
G = nx.karate_club_graph()  # 34 nodes; classic social network

net_custom = network(
    information="Raising the minimum wage will rapidly reduce unemployment.",
    # n is inferred from graph
    degree=4, rounds=8,
    depth=0.5, depth_max=4, edge_frac=0.5,
    seeds=[0,1], seed_belief=0.95, talk_prob=0.5,
    mode="llm", rng=1, model="gpt-4.1",
    graph=G,
)
net_custom.simulate()

# Plot a couple of views
net_custom.plot(type="coverage")
net_custom.plot(type="centrality", metric="degree")



In [ ]:
# Conversation retrieval examples

# Get all conversations for round 5 (dict keyed by round or list if round specified)
convos_r5 = net.conversations(round=5)
print(f"Round 5 conversations: {len(convos_r5)} edges")
if convos_r5:
    print(convos_r5[0].keys())

# Get the specific conversation between nodes 2 and 7 at round 5
rec_2_7 = net.get_conversation(round=5, u=2, v=7)
if rec_2_7:
    print("Did talk:", rec_2_7["did_talk"]) 
    # Preview first few turns
    for line in rec_2_7["turns"][:6]:
        print(line)
else:
    print("No record for (2,7) at round 5")



In [ ]:
# Summary and export examples

# Quick summary panel
print(net.summary())

# Export to files (CSV/JSONL)
# Note: comment these out if you do not want to write files in your environment.
net.export(
    history_csv="history_demo.csv",
    beliefs_csv="beliefs_demo.csv",
    conversations_jsonl="conversations_demo.jsonl",
)
print("Exported history_demo.csv, beliefs_demo.csv, conversations_demo.jsonl")

